# Generacion Features de Grafo

## Imports & Load Data

In [1]:
import warnings
warnings.filterwarnings('ignore')

In [2]:
import geopandas as gpd
import networkx as nx

import numpy as np
import pandas as pd

import pickle

In [3]:
# Cargar endpoint
infile = open('./data/1_GDB_Layers.pkl','rb')
GDB = pickle.load(infile)
infile.close()

## Generar Grafos

### Grafo General Red de Media

In [4]:
# Creamos un grafo cuyos edges son los segmentos de la red de media tension
# Asignamos pesos en base a la longitud de cada segmento
MTG = nx.Graph()
MTG.add_weighted_edges_from(GDB['SSDMT'][['PN_CON_1','PN_CON_2','COMP']].values.tolist(), weight='length')

In [5]:
# Adicionar propiedades

# Agrego el COD_ID de cada segmento como atributo de Edge
ID_attr_dict = {tuple((x[0], x[1])): x[2] for x in GDB['SSDMT'][['PN_CON_1','PN_CON_2','COD_ID']].values}
nx.set_edge_attributes(MTG, ID_attr_dict, 'SEG_ID')

In [6]:
# Correcciones manuales por inspeccion

# Agregos segmentos faltantes del grafo en base a analisis espacial
MTG.add_weighted_edges_from([['27153484','27147209', 29.9]], weight='length')
MTG.add_weighted_edges_from([['27134272','27134273', 2]], weight='length')

### Diccionario de Grafos en base al circuito de media CTMT

In [7]:
g_dict = dict()

for c in GDB['SSDMT'].CTMT.unique():
    ss_ctmt = GDB['SSDMT'][GDB['SSDMT'].CTMT == c]
    
    gg = nx.Graph()
    gg.add_weighted_edges_from(ss_ctmt[['PN_CON_1','PN_CON_2','COMP']].values.tolist(), weight='length')
    
    ID_attr_dict = {tuple((x[0], x[1])): x[2] for x in ss_ctmt[['PN_CON_1','PN_CON_2','COD_ID']].values}
    nx.set_edge_attributes(gg, ID_attr_dict, 'COD_ID')
    
    g_dict[c] = gg

In [8]:
# Agregos segmentos faltantes del grafo en base a analisis espacial
g_dict['27117490'].add_weighted_edges_from([['27153484','27147209', 29.9]], weight='length', COD_ID = 'A1')
g_dict['27117499'].add_weighted_edges_from([['27134272','27134273', 2]], weight='length', COD_ID = 'A2')

In [9]:
# Agrego el COD_ID de cada segmento como atributo de Edge
for gr in g_dict:
    nx.set_edge_attributes(g_dict[gr], ID_attr_dict, 'COD_ID')

### Definir puntos de conexion a Subestacion

In [117]:
### !!! MEJORAR EFICIENCIA DE ESTO !!!

# Las subestaciones tienen multiples puntos de llegada
# por ende definimos una lista de puntos para cada una
CTMT_SUB_PCON = pd.DataFrame(columns= ['CTMT','CTMT_PCON'])

for _, sub_poly in GDB['SUB'].iterrows():
    for i, seg_pcon in GDB['SSDMT'].iterrows():
        if sub_poly.geometry.boundary.intersects(seg_pcon.geometry) and MTG.has_node(seg_pcon.PN_CON_1):
            CTMT_SUB_PCON.loc[seg_pcon.COD_ID] = [seg_pcon.CTMT, seg_pcon.PN_CON_1]

In [118]:
# Agregar punto de conexion no encontrados
CTMT_SUB_PCON.loc['26705414'] = ['27117513', '27807944']
CTMT_SUB_PCON.loc['26705413'] = ['27117507', '27760710']

In [119]:
CTMT_SUB_PCON = CTMT_SUB_PCON[~CTMT_SUB_PCON.CTMT.duplicated()]

In [120]:
CTMT_SUB_PCON.describe()

,CTMT,CTMT_PCON
count,29,29
unique,29,25
top,27117511,27148235
freq,1,2


### Definir punto de conexion a red de media de cada Tranformador

In [121]:
# !!! MEJORAR EFICIENCIA !!!

def get_pn_con(trd_pac):
    PAC_ID = f"{trd_pac[:-2]}MT"

    if PAC_ID in GDB['SSDMT'].PAC_1.values:
        pn_con = GDB['SSDMT'][GDB['SSDMT'].PAC_1 == PAC_ID].PN_CON_1.values[0]
    elif PAC_ID in GDB['SSDMT'].PAC_2.values:
        pn_con = GDB['SSDMT'][GDB['SSDMT'].PAC_2 == PAC_ID].PN_CON_2.values[0]
    else:
        pn_con = ''
    
    return pn_con

In [122]:
UNTRD_CONECTIONS = GDB['UNTRD'][['COD_ID','PAC_1','CTMT','SUB']]

In [123]:
UNTRD_CONECTIONS['UNTRD_PCON'] = UNTRD_CONECTIONS.PAC_1.apply(get_pn_con)

C:\Users\RamiroJC\AppData\Roaming\Python\Python37\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [124]:
UNTRD_CONECTIONS.head(5)

,COD_ID,PAC_1,CTMT,SUB,UNTRD_PCON
0,26879192,75475200753159000ET,27117500,27335510,27156187
1,26880293,75333100753159000ET,27117512,27335511,27132900
2,26880295,75333800753149800ET,27117512,27335511,27137461
3,26880296,75334300753147200ET,27117512,27335511,27137303
4,26880297,75332600753159300ET,27117512,27335511,27148730


## Generar Tabla de conexiones

In [125]:
UNTRD_CONECTIONS.shape

(3848, 5)

In [127]:
UNTRD_CONECTIONS = pd.merge(UNTRD_CONECTIONS, CTMT_SUB_PCON, how='left', on='CTMT')

In [139]:
UNTRD_CONECTIONS.sample(10)

,COD_ID,PAC_1,CTMT,SUB,UNTRD_PCON,CTMT_PCON
2487,26877916,75002900754086600ET,27117490,27335509,27154833,27128870
2826,26877058,76425433751994355MT,27117484,27335513,27159475,44571767
2048,26877716,75541000753715800ET,80345444,27335511,27142033,27708175
1634,26877845,75266500753839800ET,27117490,27335509,27141205,27128870
3622,26876672,76672700753178400ET,27117484,27335513,27146517,44571767
2407,26877437,75656048753957141ET,27117486,27335509,27157771,27152908
56,26880372,75391600753158800MT,27117512,27335511,27153411,27148235
3281,26876797,76973453752623547ET,27117484,27335513,27145716,44571767
1979,26879037,75581900753351800ET,27117499,27335510,27134865,27707916
3,26880296,75334300753147200ET,27117512,27335511,27137303,27148235


## Exportar exit point

In [20]:
filename = './data/22_Full_Graph.pkl'
outfile = open(filename, 'wb')
pickle.dump(MTG,outfile)
outfile.close()

In [22]:
filename = './data/22_CTMT_Graph_Dict.pkl'
outfile = open(filename, 'wb')
pickle.dump(g_dict, outfile)
outfile.close()

In [131]:
filename = './data/22_UNTRD_Conections.pkl'
outfile = open(filename, 'wb')
pickle.dump(UNTRD_CONECTIONS, outfile)
outfile.close()